In [1]:
import cv2
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.keras as K
import tensorflow.keras.backend as Kback
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf

# from google.colab import drive
# drive.mount('/content/drive')

# DCT

In [2]:
from scipy.fftpack import dct

def dct_2d(a):
    a1 = a[0:80,0:80]
    a2 = a[0:80,80:160]
    a3 = a[80:160,0:80]
    a4 = a[80:160,80:160]
    t1 = dct(dct(a1.T).T)
    t2 = dct(dct(a2.T).T)
    t3 = dct(dct(a3.T).T)
    t4 = dct(dct(a4.T).T)
    return t1[0][0],t1[0][1],t1[1][0],t1[2][0],t1[0][2],t1[2][2],t1[0][4],t1[4][0],t1[3][2],t1[2][3],t2[0][0],t2[0][1],t2[1][0],t2[2][0],t2[0][2],t2[2][2],t2[0][4],t2[4][0],t2[3][2],t2[2][3],t3[0][0],t3[0][1],t3[1][0],t3[2][0],t3[0][2],t3[2][2],t3[0][4],t3[4][0],t3[3][2],t3[2][3],t4[0][0],t4[0][1],t4[1][0],t4[2][0],t4[0][2],t4[2][2],t4[0][4],t4[4][0],t4[3][2],t4[2][3]

# Model Training

In [3]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = Kback.sum(Kback.round(Kback.clip(y_true * y_pred, 0, 1)))
    possible_positives = Kback.sum(Kback.round(Kback.clip(y_true, 0, 1)))
    predicted_positives = Kback.sum(Kback.round(Kback.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kback.epsilon())
    recall = true_positives / (possible_positives + Kback.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+Kback.epsilon())
    return f1_val

METRICS = [
      "accuracy",
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
      f1_score
]

# MobileNetV2 all layers freezed

In [4]:
input_layer = K.Input(shape=(160,160,3))
resnet_model = K.applications.MobileNetV2(include_top = False, weights = "imagenet", input_tensor = input_layer)
for layer in resnet_model.layers:
    layer.trainable = False
for i, layer in enumerate(resnet_model.layers):
    print(i, layer.name, "-", layer.trainable)

`input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.


0 input_1 - False
1 Conv1 - False
2 bn_Conv1 - False
3 Conv1_relu - False
4 expanded_conv_depthwise - False
5 expanded_conv_depthwise_BN - False
6 expanded_conv_depthwise_relu - False
7 expanded_conv_project - False
8 expanded_conv_project_BN - False
9 block_1_expand - False
10 block_1_expand_BN - False
11 block_1_expand_relu - False
12 block_1_pad - False
13 block_1_depthwise - False
14 block_1_depthwise_BN - False
15 block_1_depthwise_relu - False
16 block_1_project - False
17 block_1_project_BN - False
18 block_2_expand - False
19 block_2_expand_BN - False
20 block_2_expand_relu - False
21 block_2_depthwise - False
22 block_2_depthwise_BN - False
23 block_2_depthwise_relu - False
24 block_2_project - False
25 block_2_project_BN - False
26 block_2_add - False
27 block_3_expand - False
28 block_3_expand_BN - False
29 block_3_expand_relu - False
30 block_3_pad - False
31 block_3_depthwise - False
32 block_3_depthwise_BN - False
33 block_3_depthwise_relu - False
34 block_3_project - Fal

# The feature map of CNN model

In [8]:
def fft_2d(feature_map):
    feature_map = tf.cast(feature_map, tf.complex64)
    X1 = tf.signal.fft2d(feature_map)
    X1 = tf.abs(X1)
    return X1   

inputs = K.layers.Input(shape=(160,160,3), name="image") 
resnet50 = resnet_model(inputs)
fft_layer = K.layers.Lambda(lambda x: fft_2d(x))(resnet50)
fft_flat = K.layers.GlobalAveragePooling2D()(resnet50)
output = K.layers.Dense(20, activation='softmax')(fft_flat)

model = K.Model(inputs=inputs, outputs=fft_flat)
optimizer = K.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model.compile(loss=["categorical_crossentropy"], metrics=METRICS, optimizer = optimizer)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 160, 160, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


c:\Python\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Data Loading

In [9]:
import os
import random
import numpy as np

Y_train = []
X_train = []
dct_train = []
for i in range(1,21):
    folder = "D:/RESEARCH/Circuit component recognition/train/"+str(i)+"/"
    for image in os.listdir(os.path.join(folder)):
        I = []
        img = cv2.imread(os.path.join(folder,image))
        img = cv2.resize(img, (160, 160))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img/255
        dct_train.append(dct_2d(img_gray))
        I.append(img)
        I = np.asarray(I)
        X_train.append(np.asarray(model.predict(I)[0]))
        Y_train.append(i)

Y_train = np.asarray(Y_train)
X_train = np.asarray(X_train)
dct_train = np.asarray(dct_train)
X_train = np.column_stack((X_train, dct_train))

Y_test = []
X_test = []
dct_test = []
for i in range(1,21):
    folder = "D:/RESEARCH/Circuit component recognition/test/"+str(i)+"/"
    for image in os.listdir(os.path.join(folder)):
        I = []
        img = cv2.imread(os.path.join(folder,image))
        img = cv2.resize(img, (160, 160))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img/255
        dct_test.append(dct_2d(img_gray))
        I.append(img)
        I = np.asarray(I)
        X_test.append(np.asarray(model.predict(I)[0]))
        Y_test.append(i)

Y_test = np.asarray(Y_test)
X_test = np.asarray(X_test)
dct_test = np.asarray(dct_test)
X_test = np.column_stack((X_test, dct_test))

shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
Y_train = Y_train[shuffler]

shuffler = np.random.permutation(len(X_test))
X_test = X_test[shuffler]
Y_test = Y_test[shuffler]

1/1 [==============================] - 0s 109ms/step


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.fit_transform(Y_test)

In [11]:
print(X_train.shape)
print(Y_train.shape)
print(dct_train.shape)

(10000, 1320)
(10000,)
(10000, 40)


# XGBoost as classsifier

In [12]:
#XGBoost as classifier using the features of CNN+FFT
import xgboost as xgb

my_model = xgb.XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
my_model.fit(X_train, Y_train)

# save to JSON
my_model.save_model("model.json")
# save to text format
my_model.save_model("model.txt")

#Load model
# model_xgb_2 = xgb.Booster()
# model_xgb_2.load_model("model.json")

c:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:37:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [13]:
#Predicting the Test set results
y_pred = my_model.predict(X_test)
 
#Making the Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)
disp = ConfusionMatrixDisplay(cm,display_labels=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])

#Accuracy
from sklearn.metrics import accuracy_score
print("Testing accuracy:")
print(accuracy_score(Y_test, y_pred))
#F1_score
from sklearn.metrics import f1_score
print("Testing F1-score")
print(f1_score(Y_test, y_pred, average = 'macro'))
#Precision
from sklearn.metrics import precision_score
print("Testing Precision:")
print(precision_score(Y_test, y_pred, average = 'macro'))
#Recall
from sklearn.metrics import recall_score
print("Testing Recall:")
print(recall_score(Y_test, y_pred, average = 'macro'))

Testing accuracy:
0.8605
Testing F1-score
0.8608743927283179
Testing Precision:
0.8684844694277789
Testing Recall:
0.8605


In [14]:
cm

array([[ 60,   6,   0,   0,   0,   0,   0,   0,   0,   1,   2,   0,   0,
          0,   0,   0,   0,   0,  31,   0],
       [  3,  71,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          3,   0,   0,   0,   0,  23,   0],
       [  0,   0,  94,   0,   0,   0,   0,   3,   0,   0,   0,   2,   0,
          0,   0,   0,   0,   0,   1,   0],
       [  0,   0,   0,  88,   6,   0,   0,   0,   0,   2,   0,   0,   3,
          0,   0,   0,   0,   0,   0,   1],
       [  0,   0,   0,   8,  92,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   1,  99,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  94,   0,   0,   0,   0,   0,   0,
          0,   0,   5,   0,   1,   0,   0],
       [  0,   0,  14,   0,   0,   0,   0,  75,   6,   0,   1,   3,   0,
          0,   0,   0,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   1,  97,   0,   0,  